In [1]:
import pandas as pd

In [2]:
# start by reading the data
df = pd.read_excel("data/Unemployment.xlsx",header=4,dtype={"FIPS_Code":str})
df.head()

,FIPS_Code,State,Area_Name,Rural_Urban_Continuum_Code_2013,Urban_Influence_Code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Civilian_labor_force_2022,Employed_2022,Unemployed_2022,Unemployment_rate_2022,Median_Household_Income_2021,Med_HH_Income_Percent_of_State_Total_2021
0,00000,US,United States,NaN,NaN,NaN,142601576.0,136904853.0,5696723.0,4.0,...,162229903.0,153544980.0,8684923.0,5.4,164781642.0,158766998.0,6014644.0,3.7,69717.0,NaN
1,01000,AL,Alabama,NaN,NaN,NaN,2147173.0,2047731.0,99442.0,4.6,...,2259349.0,2183330.0,76019.0,3.4,2286028.0,2226670.0,59358.0,2.6,53990.0,100.0
2,01001,AL,"Autauga County, AL",2.0,2.0,1.0,21861.0,20971.0,890.0,4.1,...,26545.0,25809.0,736.0,2.8,26789.0,26181.0,608.0,2.3,66444.0,123.1
3,01003,AL,"Baldwin County, AL",3.0,2.0,1.0,69979.0,67370.0,2609.0,3.7,...,99953.0,97034.0,2919.0,2.9,102849.0,100432.0,2417.0,2.4,65658.0,121.6
4,01005,AL,"Barbour County, AL",6.0,6.0,0.0,11449.0,10812.0,637.0,5.6,...,8280.0,7821.0,459.0,5.5,8241.0,7906.0,335.0,4.1,38649.0,71.6


In [3]:
# the first thing I'll do is make this just the counties of US states.  
# the easiest way to do that is by the FIPS_Code, which ends in 000 for anything that is not a county or a state
# I'll also eliminate DC and PR
df_county = df[~df.FIPS_Code.str.endswith("000") & ~df.State.isin(["DC","PR"])].copy()
# Let's print a validations that we only have 50 states
print(len(df_county.State.unique()))

50


In [4]:

# Next thing we'll do is eliminate the county/state part (or really separate and validate)
# The first thing I do is to make sure that everything after a comma is a state
area_name = df_county.Area_Name.str.split(",",expand=True)
print(area_name.loc[:,1].unique())
area_name = area_name.loc[:,0] # Eliminate the state part
# Now I think that the last thing before a comma is probably the area type
# There are several ways to get this (regex probably being the best way, but we can also use apply)
area_type = area_name.apply(lambda x: x.split()[-1])
print(area_type.unique()) # validation
# Let's eliminate these.  The Only ones I'm not clear on area the city, and area.
# Note, this was an error in the example data, though the instructions were correct
# print(area_name[area_type.str.endswith('city')]) # safe to eliminate these
# print(area_name[area_type.str.endswith('City')]) # Not safe to eliminate these "Bristol City"
# print(area_name[area_type.str.endswith('Area')]) # These should clearly be "Census Area"
to_eliminate = list(area_type.unique())
to_eliminate.remove("Area")
to_eliminate.remove("City")
to_eliminate.append("Census Area") 
print(to_eliminate) # validation
# This can be done quickly with a regex
regex = "|".join(to_eliminate) # the regex
area_name = area_name.str.replace(regex,'',regex=True)
# lastly, we'll change the original column
df_county['Area_Name'] = area_name

[' AL' ' AK' ' AZ' ' AR' ' CA' ' CO' ' CT' ' DE' ' FL' ' GA' ' HI' ' ID'
 ' IL' ' IN' ' IA' ' KS' ' KY' ' LA' ' ME' ' MD' ' MA' ' MI' ' MN' ' MS'
 ' MO' ' MT' ' NE' ' NV' ' NH' ' NJ' ' NM' ' NY' ' NC' ' ND' ' OH' ' OK'
 ' OR' ' PA' ' RI' ' SC' ' SD' ' TN' ' TX' ' UT' ' VT' ' VA' ' WA' ' WV'
 ' WI' ' WY']
['County' 'Borough' 'Area' 'Borough/municipality' 'Borough/city'
 'Municipality' 'County/city' 'Parish' 'city' 'County/town' 'City']
['County', 'Borough', 'Borough/municipality', 'Borough/city', 'Municipality', 'County/city', 'Parish', 'city', 'County/town', 'Census Area']


In [5]:
# Next thing will be the eliminate some of the duplicate data by melting the data
melt_columns = df_county.columns[df_county.columns.str.startswith("Unemployed") | df_county.columns.str.startswith("Employed")]
index_columns = ['FIPS_Code','State','Area_Name','Rural_Urban_Continuum_Code_2013','Urban_Influence_Code_2013','Metro_2013','Median_Household_Income_2021']
melted = df_county.melt(id_vars=index_columns,value_vars=melt_columns)
# now the "variable" column contains both the type and year, so we need to split it
melted[["Employment","Year"]] = melted.variable.str.split("_",expand=True)
# finally, we can repivot to recover the data for each county's year
df_final = melted.pivot(index=list(index_columns)+["Year"],columns="Employment",values="value").reset_index().copy()
# But, this will contain some NAs, because we don't have data for some years, so we drop them
df_final.dropna(inplace=True)
df_final.to_csv('data/county_unemployment.csv',index=False)